In [56]:
import numpy as np

In [19]:
# Recursive implementation

class RecursiveDFS():
    
    def __init__(self, graph):
        """
        Args:
        graph (dict): implementation of graph data structure via dictionary of adjacency list
        """
        self.graph = graph
        self.explored = {key : False for key in graph.keys()} # initialize all node to unexplored

    def dfs(self, source):
        """
        Args:
        source (str): node for which we want to explore all findable nodes
        """
        self.explored[source] = True

        for neighbour in graph[source]:
            if not self.explored[neighbour]:
                self.dfs(neighbour)

# Driver code
graph = {'a': ['b', 'c'],
        'b': ['c', 'd'],
        'c': ['a', 'b'],
        'd': ['b'],
        'e': ['f', 'g'],
        'f': ['e', 'g'],
        'g': ['e', 'f']}

DFS = RecursiveDFS(graph)
DFS.dfs('f')

print(DFS.explored)

In [30]:
# Substitute queue with stack from BFS implementation
# Non recursive implementation

class DFSearcher():
    
    def __init__(self, graph):
        """
        Args:
        graph (dict): implementation of graph data structure via dictionary of adjacency list
        """
        self.graph = graph
        self.explored = {key : False for key in graph.keys()} # initialize all node to unexplored

    def dfs(self, source):
        """
        Args:
        source (str): node for which we want to explore all findable nodes
        """
        self.explored[source] = True

        stack = []
        stack.append(source)

        while stack: # not empty
            current_node = stack[-1]
            stack.pop(-1)

            for neighbour in graph[current_node]:
                if not self.explored[neighbour]:
                    self.explored[neighbour] = True
                    stack.append(neighbour)

                    
# Driver code
graph = {'a': ['b', 'c'],
        'b': ['c', 'd'],
        'c': ['a', 'b'],
        'd': ['b'],
        'e': ['f', 'g'],
        'f': ['e', 'g'],
        'g': ['e', 'f']}

DFS = DFSearcher(graph)
DFS.dfs('a')

print(DFS.explored)

__Topological Ordering__

In [117]:
class Tree(): 
    # Represent a tree node
    def __init__(self, root, children = []):
        self.children = children
        self.root = root

    def add_child(self, child): # child is of type Tree 
        self.children.append(child)
        
    def print_paths(self, path):
        
        path.append(self.root)
        
        if not self.children:
            print(path) 
        
        for child in self.children:
            child.print_paths(path) # why changes are mantained?
        
        path.pop(-1)

In [118]:
# Tree driver code

my_tree = Tree('a', [Tree('b', [Tree('d'), Tree('e')]), \
                    Tree('c',  [Tree('f')])])

# my_tree = Tree('a', [Tree('b')])

my_tree.print_paths([])

['a', 'b', 'd']
['a', 'b', 'e']
['a', 'c', 'f']


In [44]:
# Recursive implementation

class TopologicalSorter():
    
    def __init__(self, graph):
        """
        Args:
        graph (dict): implementation of graph data structure via dictionary of adjacency list
        """
        self.graph = graph
        self.explored = {key : False for key in graph.keys()} # initialize all node to unexplored
        self.topological_order = []
        self.current_label = len(list(graph.keys()))

    def dfs(self, source):
        """
        Args:
        source (str): node for which we want to explore all findable nodes
        """
        self.explored[source] = True

        for neighbour in graph[source]:
            if not self.explored[neighbour]:
                self.dfs(neighbour)
                
        self.topological_order.append((source, self.current_label))
        self.current_label -= 1
        
    
    def sort(self):
        
        for vertex in self.graph.keys():
            if not self.explored[vertex]:
                self.dfs(vertex)
                
        self.topological_order = [self.topological_order[i] \
                                  for i in reversed(range(len(self.topological_order)))]

# Driver code
graph = {'a': ['b', 'c'],
        'b': ['d'],
        'c': ['b'],
        'd': [],
        }

ts = TopologicalSorter(graph)
ts.sort()

print(ts.topological_order) 

[('a', 1), ('c', 2), ('b', 3), ('d', 4)]


In [119]:
# Need to handle multiple paths

[('a', 1), ('c', 2), ('b', 3), ('d', 4)]


__Kosaraju Algorithm__  
Count number of SCCs in directed graph

In [62]:
class Kosaraju():
    
    def reverse_graph(self, graph):
        reversed_graph = {key : [] for key in graph.keys()}
        for key, values in graph.items():
            # Iterate over the values, and add / append key to the value list
            for val in values:
                reversed_graph[val].append(key)
        
        return reversed_graph
        
        
    def dfs_loop(self, graph):
        
        # Initialization
        self.source = None 
        self.time = 0 
        self.explored = {key : False for key in graph.keys()}
        self.leaders = dict() # define common leader node of a SCC
        
        # Hp: starting node is 1
        for node in reversed(range(1, len(graph)+1)):
            if not self.explored[node]:
                self.source = node
                self.dfs(graph, node)
                
    
    def dfs(self, graph, node):
        self.explored[node] = True
        
        # update SCC leader
        try:
            self.leaders[self.source].append(node)
        except:
            self.leaders[self.source] = [node]
        
        for edge_node in graph[node]:
            if not self.explored[edge_node]:
                self.dfs(graph, edge_node)
                
        self.time += 1
        self.finishing_times[node] = self.time
        
    def map_graph(self, graph):
        """
        Map graph nodes and edges with values in self.finishing_times
        """
        
        mapped_graph = dict()
        
        for node, edges in graph.items():
            mapped_node = self.finishing_times[node]
            mapped_edges = []
            for e in edges:
                mapped_edges.append(self.finishing_times[e])
            
            mapped_graph[mapped_node] = mapped_edges
        
        return mapped_graph
        
        
    def count_scc(self, graph):
        # Initialization
        self.finishing_times = dict()
        
        reversed_graph = self.reverse_graph(graph)
        
        # First pass
        self.dfs_loop(reversed_graph)
        
        # Map graph with finishing times
        mapped_graph = self.map_graph(graph)
        
        # Second pass
        self.dfs_loop(mapped_graph)
        
        return len(self.leaders)
        

# Driver code
graph = {1: [2, 3],
        2: [3, 4],
        3: [1, 2],
        4: [],
        5: [6, 7],
        6: [5, 7],
        7: [5, 6]}


scc_counter = Kosaraju()
num_scc = scc_counter.count_scc(graph)

print(f"The number of SCCs for the input graph is {num_scc}")

{6: 4, 5: 2, 7: 1, 3: 7, 1: 5, 2: 6, 4: 3}
{5: [6, 4], 6: [4, 7], 4: [5, 6], 7: [], 2: [1, 3], 1: [2, 3], 3: [2, 1]}
{7: [7], 6: [6, 4, 5], 3: [3, 2, 1]}
The number of SCCs for the input graph is 3
